In [2]:
# Set constants for environment
import os
import sys

PROJECT_NAME = 'lvgl-ui-detector'

IN_COLAB = 'google.colab' in sys.modules
IN_LOCAL_CONTAINER = 'HOMELAB' in os.environ
if IN_COLAB:
    from google.colab import drive
    drive.mount('/content/drive/project') # Mount Google Drive
    root_dir = os.path.join("/content/drive/project")
elif IN_LOCAL_CONTAINER:
    root_dir = os.path.join(os.environ['HOME'], 'project')
else:
    root_dir = os.path.curdir

In [5]:
if IN_COLAB and 'ultralytics' not in sys.modules:
    %pip install ultralytics
%pip show ultralytics

Note: you may need to restart the kernel to use updated packages.


# Note on using tensorboard
`! tensorboard --logdir ${ULTRALYTICS_DIR}/runs --port 6006 --host 0.0.0.0 # needs '--host 0.0.0.0'`

# Data preparation

In [6]:
import glob
if IN_COLAB:
    repository_dir = os.path.join(root_dir, PROJECT_NAME)
    dataset_dir = os.path.join(root_dir, 'datasets')
    dataset_files = glob.glob(os.path.join(dataset_dir, '*.yaml'))
    print(f"Datasets: {dataset_files}")
    # The following command will fail if the repository does not exist
    % cd repository_dir && git pull
else:
    datasets_dir = os.path.join(root_dir, 'tmp')
    dataset_files = glob.glob(os.path.join(datasets_dir, '*.yaml'))
    print(f"Datasets: {dataset_files}")

Datasets: ['./tmp/designs.yaml', './tmp/random_widgets.yaml', './tmp/random_uis.yaml']


# Model training

In [ ]:
from ultralytics import YOLO
model = YOLO('yolov8n.pt')

results = model.train(data=f"/workspace/project/tmp/random_widgets.yaml", epochs=100, imgsz=480, device=[0])

In [ ]:
# Perform validation using the validate function
model = YOLO('/usr/src/ultralytics/runs/detect/train7/weights/best.pt')
results = model.val()

In [ ]:
results.box.map

In [ ]:
results.box.map50

In [ ]:
results.box.map75

In [ ]:
results.box.maps

In [ ]:
from ultralytics.utils.benchmarks import benchmark

# Benchmark on GPU
benchmark(model='/usr/src/ultralytics/runs/detect/train7/weights/best.pt', data='/workspace/project/tmp/custom/custom.yaml', imgsz=416, half=False, device=[0])